In [ ]:
import torch
from pyannote.database import get_protocol, FileFinder

preprocessors = {'audio': FileFinder()}
protocol = get_protocol('VOXCON.SpeakerDiarization.Sample', preprocessors=preprocessors)

pipeline = torch.hub.load('pyannote/pyannote-audio', 'dia_dihard')

In [ ]:
test_file = next(protocol.test())

diarization = pipeline(test_file)

In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate

metric = DiarizationErrorRate()
groundtruth = test_file["annotation"]

der = metric(groundtruth, diarization)

print(f'diarization error rate = {100 * der:.1f}%')

In [ ]:
diarization

In [ ]:
groundtruth

In [ ]:
from pyannote.database.util import load_rttm
rttm = load_rttm('/home/jovyan/work/uis-rnn-voxsrc20/lists/voxconverse/sample.rttm')[test_file['uri']]
test_file['uri']

In [ ]:
from pyannote.audio.features import RawAudio
from pyannote.core import Segment, notebook
from IPython.display import Audio

# make notebook visualization zoom on 600s < t < 660s time range
# EXCERPT = Segment(600, 660)
# notebook.crop = EXCERPT

# visualize excerpt groundtruth
groundtruth

file = {'uri': test_file['uri'], 'audio': test_file['audio']}

# load audio waveform, crop excerpt, and play it
waveform = RawAudio(sample_rate=16000)(file).data
Audio(data=waveform.squeeze(), rate=16000, autoplay=True)

In [ ]:
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f'Speaker "{speaker}" speaks between t={turn.start:.1f}s and t={turn.end:.1f}s.')

mapping = metric.optimal_mapping(groundtruth, diarization)
diarization.rename_labels(mapping=mapping)